# gnina docking

## Setup

In [ ]:
#| default_exp gnina_docking

In [ ]:
#| export
from rdkit import Chem
from rdkit.Chem import AllChem
import subprocess
import os
import py3Dmol
import re
# from tools.dataset import Data
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns

### Uncheck below when running a new notebook

In [ ]:
# !sudo apt install -yq openbabel

# !wget https://github.com/gnina/gnina/releases/download/v1.0.3/gnina #check the latest version please

# !chmod +x gnina

# !./gnina

## Get protein and ligand

In [ ]:
# #nature medicine paper, mrtx
# !wget http://files.rcsb.org/download/7T47.pdb

# # get ligand
# !grep 6IC 7T47.pdb > lig.pdb

# # get protein
# # !grep ATOM 7T47.pdb > rec.pdb #not as good as the one below

# !grep -v 6IC 7T47.pdb > rec.pdb

## Conformer generation

In [ ]:
#| export
def rdkit_conformer(SMILES, # SMILES string
                    output, # file ".sdf" to be saved
                    method='ETKDG', # Optimization method, can be 'UFF', 'MMFF' or 'ETKDGv3'
                    visualize=True, #whether or not to visualize the compound
                    seed = 3,# randomness of the 3d
                   ):
    
    np.random.seed(seed) 
    # Generate a 2D molecule from the SMILES string
    mol = Chem.MolFromSmiles(SMILES)
    
    # Generate a 3D conformation of the molecule
    AllChem.EmbedMolecule(mol)
    

    # Optimize the 3D conformation using the specified force field method
    if method == 'UFF':
        AllChem.UFFOptimizeMolecule(mol)
    elif method == 'MMFF':
        AllChem.MMFFOptimizeMolecule(mol)
    elif method == 'ETKDG':
        AllChem.EmbedMultipleConfs(mol, numConfs=1, useExpTorsionAnglePrefs=True, 
                                   useBasicKnowledge=True, enforceChirality=True,randomSeed=seed)
        AllChem.ETKDGv3()
        AllChem.UFFOptimizeMolecule(mol)
        
#     elif method == 'ETKDG_MMFF':
#         AllChem.EmbedMultipleConfs(mol, numConfs=1, useExpTorsionAnglePrefs=True, 
#                                    useBasicKnowledge=True, enforceChirality=True)
#         AllChem.ETKDGv3()
#         AllChem.MMFFOptimizeMolecule(mol)
    else:
        raise ValueError('Invalid method specified')
        

    # Remove hydrogens from the molecule
    # mol = Chem.RemoveHs(mol)
    
    # create the directory if it does not exist
    directory = os.path.dirname(os.path.abspath(output))
    os.makedirs(directory, exist_ok=True)

    # Write the molecule to an SDF file
    w = Chem.SDWriter(output)
    w.write(mol)
    w.close()
    
    if visualize:
        v = py3Dmol.view()
        v.addModel(Chem.MolToMolBlock(mol))
        v.setStyle({'stick':{}})
        v.zoomTo()
        v.show()

## Gnina affinity

In [ ]:
#| export
def extract_values(gnina_output):
    "Extract values from gnina output"
    # Search for the line containing mode 1
    mode1_line = re.search(r'\b1\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\b', gnina_output)
    
    if mode1_line:
        # Extract the values from the matched line
        affinity = float(mode1_line.group(1))
        cnn_pose_score = float(mode1_line.group(2))
        cnn_affinity = float(mode1_line.group(3))
        
        return affinity, cnn_pose_score, cnn_affinity
    
    return None

In [ ]:
#| export
def run_gnina(receptor, # receptor file
              ligand, # ligand file
              autobox_ligand, # ligand file isolated from the complex
              output = 'docked.sdf', # output file (sdf or sdf.gz) to be saved
              seed=0, # set seeds
              exhaustiveness=None, # number of MC chains, default is 8 if None, the higher the better (16,32); for whole protein, use 64
              ):
    
    command = ['./gnina', 
               '-r', receptor, 
               '-l', ligand, 
               '--autobox_ligand', autobox_ligand,
               '-o', output,
               '--seed', str(seed)]

    if exhaustiveness is not None:
        command.extend(['--exhaustiveness', str(exhaustiveness)])

    # Run the command and capture the output
    result = subprocess.run(command, capture_output=True, text=True)
    
    print(f'save the docked file as {output}')
    
    values = extract_values_from_output(result.stdout)
    
    print(f'affinity, cnn_pose_score, and cnn_affinity are: {values}')

    return values

In [ ]:
# df = Data.get_g12d_dedup()[:10]

In [ ]:
#| export
def run_gnina_df(df, 
                 ID_col = 'ID',
                 smi_col = 'SMILES',
                 output_df = 'gnina.csv',
                 ):
    affinity_values = []
    cnn_pose_score_values = []
    cnn_affinity_values = []

    # Create a tqdm progress bar
    progress_bar = tqdm(total=len(df), desc='Processing')
    
    # create the directory if it does not exist
    os.makedirs('docked', exist_ok=True)

    for i, r in df.iterrows():
        rdkit_conformer(SMILES=r[smi_col], output = f'ligand/{r[ID_col]}.sdf', visualize=False)
        affinity, cnn_pose_score, cnn_affinity = run_gnina('rec.pdb',f'ligand/{r[ID_col]}.sdf', 'lig.pdb',f'docked/docked_{r[ID_col]}.sdf')

        affinity_values.append(affinity)
        cnn_pose_score_values.append(cnn_pose_score)
        cnn_affinity_values.append(cnn_affinity)

        # Update the progress bar
        progress_bar.update(1)

    # Close the progress bar
    progress_bar.close()

    # Assign the calculated values as new columns in the DataFrame
    df = df.copy()
    df['Affinity'] = affinity_values
    df['CNN_Pose_Score'] = cnn_pose_score_values
    df['CNN_Affinity'] = cnn_affinity_values
    
    df.to_csv(output_df,index=False)
    
    return df

In [ ]:
# out = run_gnina_df(df)

## End

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()